<a href="https://colab.research.google.com/github/MaiKhoa0101/MachineLearning/blob/main/Subtitle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install -q flask flask-cors pyngrok
!pip install -q torch transformers accelerate librosa soundfile

# FFmpeg đã có sẵn trong Colab
!ffmpeg -version | head -n 1

print("\n✅ Đã cài đặt xong tất cả dependencies!")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers

✅ Đã cài đặt xong tất cả dependencies!


In [ ]:
# Lấy ngrok authtoken miễn phí tại: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "37keNPVBOMSK3Xv0QVQdp9LTmWe_6As1mhkX6PWKriBGnsRhb"

"""
Flask API cho Google Colab - Tạo phụ đề tiếng Việt
HỖ TRỢ: File Upload, URL (JSON), URL (Form-data), Google Drive Links
"""

from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from werkzeug.utils import secure_filename
import os
import subprocess
from pathlib import Path
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa
import tempfile
import uuid
from datetime import datetime
import logging
from pyngrok import ngrok, conf
import requests
import re

# Setup ngrok
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
CORS(app)

# Configuration
app.config['MAX_CONTENT_LENGTH'] = 500 * 1024 * 1024  # 500MB
app.config['UPLOAD_FOLDER'] = '/content/uploads'
app.config['OUTPUT_FOLDER'] = '/content/outputs'
app.config['ALLOWED_EXTENSIONS'] = {'mp4', 'avi', 'mov', 'mkv', 'flv', 'wmv', 'webm'}

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['OUTPUT_FOLDER'], exist_ok=True)

# Load model
logger.info("Loading PhoWhisper model...")
MODEL_NAME = "vinai/PhoWhisper-medium"
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

processor = WhisperProcessor.from_pretrained(MODEL_NAME)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to(device)
logger.info("Model loaded successfully!")


class VideoSubtitleGenerator:
    def __init__(self, model, processor, device):
        self.model = model
        self.processor = processor
        self.device = device

    def extract_audio(self, video_path):
        audio_path = tempfile.mktemp(suffix='.wav')
        command = [
            'ffmpeg', '-i', video_path,
            '-vn', '-acodec', 'pcm_s16le',
            '-ar', '16000', '-ac', '1',
            '-y', audio_path
        ]
        subprocess.run(command, check=True, capture_output=True, timeout=300)
        return audio_path

    def transcribe_audio(self, audio_path, chunk_length_s=30):
        logger.info(f"Loading audio...")
        audio, sr = librosa.load(audio_path, sr=16000)
        duration = len(audio) / sr

        chunk_samples = chunk_length_s * 16000
        segments = []
        total_chunks = (len(audio) + chunk_samples - 1) // chunk_samples

        for idx, i in enumerate(range(0, len(audio), chunk_samples)):
            chunk = audio[i:i + chunk_samples]
            logger.info(f"Processing chunk {idx+1}/{total_chunks}")

            inputs = self.processor(chunk, sampling_rate=16000, return_tensors="pt")
            input_features = inputs.input_features.to(self.device)

            with torch.no_grad():
                predicted_ids = self.model.generate(
                    input_features,
                    language="vi",
                    task="transcribe"
                )

            text = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

            if text.strip():
                start_time = i / 16000
                end_time = min((i + len(chunk)) / 16000, duration)
                segments.append({
                    'start': start_time,
                    'end': end_time,
                    'text': text.strip()
                })

        return segments

    def format_timestamp(self, seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        millis = int((seconds % 1) * 1000)
        return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

    def create_srt(self, segments, output_path):
        with open(output_path, 'w', encoding='utf-8') as f:
            for i, segment in enumerate(segments, 1):
                f.write(f"{i}\n")
                f.write(f"{self.format_timestamp(segment['start'])} --> ")
                f.write(f"{self.format_timestamp(segment['end'])}\n")
                f.write(f"{segment['text']}\n\n")
        return output_path

    def process_video(self, video_path, output_dir):
        audio_path = None
        try:
            logger.info("Extracting audio...")
            audio_path = self.extract_audio(video_path)

            logger.info("Transcribing audio...")
            segments = self.transcribe_audio(audio_path)

            video_name = Path(video_path).stem
            srt_path = os.path.join(output_dir, f"{video_name}.srt")
            self.create_srt(segments, srt_path)

            txt_path = os.path.join(output_dir, f"{video_name}_transcript.txt")
            full_text = ' '.join([s['text'] for s in segments])
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write(full_text)

            return {
                'srt_path': srt_path,
                'txt_path': txt_path,
                'segments': segments,
                'full_text': full_text
            }
        finally:
            if audio_path and os.path.exists(audio_path):
                try:
                    os.remove(audio_path)
                except:
                    pass


generator = VideoSubtitleGenerator(model, processor, device)


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']


def convert_google_drive_link(url):
    """Chuyển Google Drive sharing link thành direct download link"""
    match = re.search(r'/file/d/([a-zA-Z0-9_-]+)', url)
    if match:
        file_id = match.group(1)
        return f"https://drive.google.com/uc?export=download&id={file_id}"

    match = re.search(r'[?&]id=([a-zA-Z0-9_-]+)', url)
    if match:
        file_id = match.group(1)
        return f"https://drive.google.com/uc?export=download&id={file_id}"

    return url


def download_video_from_url(url, save_path):
    """Tải video từ URL về local, hỗ trợ Google Drive"""
    logger.info(f"Downloading video from URL: {url}")

    # Convert Google Drive link
    if 'drive.google.com' in url:
        original_url = url
        url = convert_google_drive_link(url)
        logger.info(f"Converted Google Drive link: {url}")

    # Download
    session = requests.Session()
    response = session.get(url, stream=True, timeout=300, allow_redirects=True)

    # Handle Google Drive virus warning
    if 'drive.google.com' in url and 'virus scan warning' in response.text.lower():
        logger.info("Handling Google Drive virus warning...")
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                url += f"&confirm={value}"
                response = session.get(url, stream=True, timeout=300)
                break

    response.raise_for_status()

    # Validate content type
    content_type = response.headers.get('content-type', '').lower()
    if 'text/html' in content_type:
        raise ValueError(
            "URL returned HTML instead of video file. "
            "If this is a Google Drive link, ensure the file is set to 'Anyone with the link can view'"
        )

    # Save file
    with open(save_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    # Validate file size
    file_size = os.path.getsize(save_path)
    logger.info(f"Downloaded successfully ({file_size / 1024 / 1024:.2f} MB)")

    if file_size < 1024:
        raise ValueError("Downloaded file is too small (likely an error page)")

    return save_path


@app.route('/', methods=['GET'])
def index():
    return jsonify({
        'service': 'Video Subtitle Generator API',
        'version': '2.0.0',
        'model': MODEL_NAME,
        'device': device,
        'status': 'running'
    })


@app.route('/api/phowhisper/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'healthy',
        'model_loaded': True,
        'device': device,
        'timestamp': datetime.now().isoformat()
    })


@app.route('/api/phowhisper/generate-subtitle', methods=['POST'])
def generate_subtitle():
    """
    Hỗ trợ 3 input methods:
    1. JSON body: {"url": "VIDEO_URL"}
    2. Form-data: url=VIDEO_URL
    3. Form-data: video=[file]
    """
    request_id = str(uuid.uuid4())
    video_path = None
    video_url = None

    logger.info(f"Received request: {request_id}")
    try:
        # Method 1: JSON body
        if request.is_json:
            data = request.get_json()
            video_url = data.get('url') or data.get('videoUrl')
            if video_url:
                logger.info(f"[JSON] Received URL: {video_url}")

        # Method 2: Form-data URL
        elif request.form.get('url'):
            video_url = request.form.get('url')
            logger.info(f"[FORM] Received URL: {video_url}")

        # Process URL
        if video_url:
            ext = 'mp4'
            if '.' in video_url.split('/')[-1]:
                ext = video_url.split('.')[-1].split('?')[0].lower()
                if ext not in app.config['ALLOWED_EXTENSIONS']:
                    ext = 'mp4'

            filename = f"{request_id}_from_url.{ext}"
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            download_video_from_url(video_url, video_path)

        # Method 3: File upload
        elif 'video' in request.files:
            file = request.files['video']

            if file.filename == '':
                return jsonify({'error': 'No file selected'}), 400

            if not allowed_file(file.filename):
                return jsonify({'error': 'Invalid file format'}), 400

            filename = secure_filename(file.filename)
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], f"{request_id}_{filename}")
            file.save(video_path)
            logger.info(f"[FILE] Received: {filename}")

        # No input provided
        else:
            return jsonify({
                'error': 'No video provided',
                'accepted_formats': [
                    'JSON: {"url": "VIDEO_URL"}',
                    'Form-data: url=VIDEO_URL',
                    'Form-data: video=[file]'
                ]
            }), 400

        # Process video
        output_dir = os.path.join(app.config['OUTPUT_FOLDER'], request_id)
        os.makedirs(output_dir, exist_ok=True)

        logger.info(f"Processing video...")
        result = generator.process_video(video_path, output_dir)

        # Cleanup
        if video_path and os.path.exists(video_path):
            os.remove(video_path)

        # Return result
        return_type = request.form.get('return_type') or (request.get_json() or {}).get('return_type', 'json')

        if return_type == 'file':
            return send_file(
                result['srt_path'],
                as_attachment=True,
                download_name=f"subtitle_{request_id}.srt"
            )

        return jsonify({
            'success': True,
            'request_id': request_id,
            'source': 'url' if video_url else 'upload',
            'total_segments': len(result['segments']),
            'device_used': device,
            'downloads': {
                'srt': f"/api/phowhisper/download/{request_id}/srt",
                'txt': f"/api/phowhisper/download/{request_id}/txt"
            },
            'preview': {
                'first_3_segments': result['segments'][:3],
                'text_preview': result['full_text'][:200] + '...' if len(result['full_text']) > 200 else result['full_text']
            }
        }), 200

    except requests.exceptions.RequestException as e:
        logger.error(f"Download error: {str(e)}")
        return jsonify({
            'success': False,
            'error': f'Failed to download video: {str(e)}',
            'hint': 'For Google Drive, ensure file is set to "Anyone with the link can view"'
        }), 400

    except Exception as e:
        logger.error(f"Error: {str(e)}", exc_info=True)
        if video_path and os.path.exists(video_path):
            try:
                os.remove(video_path)
            except:
                pass

        return jsonify({
            'success': False,
            'error': str(e)
        }), 500


@app.route('/api/phowhisper/download/<request_id>/<file_type>', methods=['GET'])
def download_file(request_id, file_type):
    if file_type not in ['srt', 'txt']:
        return jsonify({'error': 'Invalid file type'}), 400

    output_dir = os.path.join(app.config['OUTPUT_FOLDER'], request_id)

    if not os.path.exists(output_dir):
        return jsonify({'error': 'Request not found'}), 404

    if file_type == 'srt':
        files = list(Path(output_dir).glob('*.srt'))
    else:
        files = list(Path(output_dir).glob('*_transcript.txt'))

    if not files:
        return jsonify({'error': 'File not found'}), 404

    return send_file(
        str(files[0]),
        as_attachment=True,
        download_name=Path(files[0]).name
    )


if __name__ == '__main__':
    ngrok.kill()
    public_url = ngrok.connect(5020)

    print(f"\n{'='*70}")
    print("✅ PHOWHISPER API SERVER RUNNING")
    print(f"{'='*70}")
    print(f"🌍 PUBLIC URL: {public_url}")
    print(f"\n📖 USAGE:")
    print(f"\n   Method 1 - JSON (Recommended):")
    print(f"   POST {public_url}/api/phowhisper/generate-subtitle")
    print(f"   Content-Type: application/json")
    print(f'   Body: {{"url": "VIDEO_URL"}}')
    print(f"\n   Method 2 - Form-data URL:")
    print(f"   POST {public_url}/api/phowhisper/generate-subtitle")
    print(f"   Body: form-data, key=url, value=VIDEO_URL")
    print(f"\n   Method 3 - File Upload:")
    print(f"   POST {public_url}/api/phowhisper/generate-subtitle")
    print(f"   Body: form-data, key=video, value=[file]")
    print(f"\n💡 Supports: Direct URLs, Google Drive links")
    print(f"{'='*70}\n")

    app.run(host='0.0.0.0', port=5020, debug=False)


✅ PHOWHISPER API SERVER RUNNING
🌍 PUBLIC URL: NgrokTunnel: "https://chummiest-raiden-unimmanent.ngrok-free.dev" -> "http://localhost:5020"

📖 USAGE:

   Method 1 - JSON (Recommended):
   POST NgrokTunnel: "https://chummiest-raiden-unimmanent.ngrok-free.dev" -> "http://localhost:5020"/api/phowhisper/generate-subtitle
   Content-Type: application/json
   Body: {"url": "VIDEO_URL"}

   Method 2 - Form-data URL:
   POST NgrokTunnel: "https://chummiest-raiden-unimmanent.ngrok-free.dev" -> "http://localhost:5020"/api/phowhisper/generate-subtitle
   Body: form-data, key=url, value=VIDEO_URL

   Method 3 - File Upload:
   POST NgrokTunnel: "https://chummiest-raiden-unimmanent.ngrok-free.dev" -> "http://localhost:5020"/api/phowhisper/generate-subtitle
   Body: form-data, key=video, value=[file]

💡 Supports: Direct URLs, Google Drive links

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5020
 * Running on http://172.28.0.12:5020
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import threading
import time
from pyngrok import ngrok

print("="*60)
print("🚀 STARTING FLASK API SERVER")
print("="*60)

# Kill existing ngrok tunnels
ngrok.kill()

# Start Flask in background thread
def run_flask():
    import flask_api_colab
    flask_api_colab.app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Wait for Flask to start
print("\n⏳ Waiting for Flask to start...")
time.sleep(10)  # Tăng thời gian chờ lên 10 giây

# Start ngrok tunnel
print("\n🌐 Creating ngrok tunnel...")
public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "="*60)
print("✅ SERVER IS RUNNING!")
print("="*60)
print(f"\n🌍 PUBLIC URL: {public_url}")
print(f"\n📮 SỬ DỤNG TRONG POSTMAN:")
print(f"   Method: POST")
print(f"   URL: {public_url}/api/phowhisper/generate-subtitle")
print(f"   Body: form-data")
print(f"   Key: video (File)")
print(f"   Value: [chọn file video]")
print("\n" + "="*60)

# Keep the cell running
try:
    print("\n🟢 Server đang chạy... (Nhấn Stop để dừng)\n")
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Stopping server...")
    ngrok.kill()

In [ ]:
import requests
import json

# Thay YOUR_NGROK_URL bằng URL từ cell trên
BASE_URL = ""  # Ví dụ: https://xxxx.ngrok-free.app

# Test 1: Health check
print("🔍 Testing health endpoint...")
response = requests.get(f"{BASE_URL}/api/health")
print(json.dumps(response.json(), indent=2))

# Test 2: API info
print("\n🔍 Testing info endpoint...")
response = requests.get(f"{BASE_URL}/")
print(json.dumps(response.json(), indent=2))